In [51]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import xgboost as xgb

In [52]:
train_data = pd.read_csv('train_data.csv', header=None)

In [53]:
#将所有object转换为float
train_data = train_data.astype(float)

In [54]:
num_columns = train_data.shape[1]
print("num_columns: ", num_columns)

num_columns:  64


In [55]:
#给dataframe加上列名
column_names = []
for i in range(num_columns-1):
    column_names.append(f'{i//3}{"xyz"[i%3]}')
column_names.append('label')

train_data.columns = column_names

In [56]:
test_data = train_data.sample(frac=0.5)  # 例如抽样比例为0.5，即抽样50%的数据
train_data = train_data.drop(test_data.index)  # 删除抽样的数据

In [57]:
X = train_data.iloc[:train_data.shape[0], :]
y = train_data['label']

Xval = test_data.iloc[:test_data.shape[0], :]
yval = test_data['label']


print(X.shape, y.shape, Xval.shape, yval.shape)

(714, 64) (714,) (714, 64) (714,)


In [58]:
#统计各类别的数量
print(y.value_counts())
print(yval.value_counts())

0.0    369
1.0    345
Name: label, dtype: int64
0.0    391
1.0    323
Name: label, dtype: int64


In [59]:
xg_classifier = xgb.XGBClassifier(
            objective='multi:softprob',
            num_class = 3,
            tree_method="hist",
            n_estimators=3000,
            learning_rate=0.01,
            reg_lambda = 0.2,
            reg_alpha = 0.2,
            max_leaves = 5,
            subsample=0.7,
            colsample_bytree=0.15,
            max_bin=4096,
            n_jobs=-1,
            early_stopping_rounds=100,
            random_state=2020
        )

In [60]:
xg_classifier.fit(X, y, eval_set=[(Xval, yval)], verbose=100)

[0]	validation_0-mlogloss:1.09239
[100]	validation_0-mlogloss:0.60785
[200]	validation_0-mlogloss:0.38845
[300]	validation_0-mlogloss:0.26368
[400]	validation_0-mlogloss:0.19385
[500]	validation_0-mlogloss:0.14520
[600]	validation_0-mlogloss:0.11432
[700]	validation_0-mlogloss:0.09558
[800]	validation_0-mlogloss:0.07634
[900]	validation_0-mlogloss:0.06402
[1000]	validation_0-mlogloss:0.05531
[1100]	validation_0-mlogloss:0.04595
[1200]	validation_0-mlogloss:0.03893
[1300]	validation_0-mlogloss:0.03306
[1400]	validation_0-mlogloss:0.02774
[1500]	validation_0-mlogloss:0.02380
[1600]	validation_0-mlogloss:0.02086
[1700]	validation_0-mlogloss:0.01827
[1800]	validation_0-mlogloss:0.01574
[1900]	validation_0-mlogloss:0.01389
[2000]	validation_0-mlogloss:0.01248
[2100]	validation_0-mlogloss:0.01083
[2200]	validation_0-mlogloss:0.00984
[2300]	validation_0-mlogloss:0.00877
[2400]	validation_0-mlogloss:0.00784
[2500]	validation_0-mlogloss:0.00717
[2600]	validation_0-mlogloss:0.00666
[2700]	valida

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.15, early_stopping_rounds=100,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=4096,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=5,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=3000, n_jobs=-1, num_class=3, num_parallel_tree=None,
              objective='multi:softprob', ...)

In [61]:
#保存成.model模型
xg_classifier.save_model('xgboost_model.model')